In [1]:
# imports
from typing import List

import re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# constants
VALID_CHAR_REGEX = "[A-Z0-9]"
VALID_CHAR_CZECH_REGEX = "[a-zA-ZáčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ]" # czech lang
VALID_CHAR_POLISH_REGEX = "[a-zA-ZąćęłńóśżźĄĆĘŁŃÓŚŻŹ]" # polish lang
VALID_CHAR_GERMAN_REGEX = "[a-zA-ZÄäÖöÜüẞß]" # german lang
VALID_CHAR_ENGLISH_REGEX = "[a-zA-Z]"
INVALID_CHAR_REGEX = "[.,()«»?!-—:;…]"

VALID_CHAR_PUNC_REGEX = "[a-zA-Z0-9/=ąćęłńóśżźĄĆĘŁŃÓŚŻŹáčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽÄäÖöÜüẞß]"
PUNCT_REGEX = "[=]"

In [3]:
# update these values to change text input for the program
WOJNICZ_INPUT_FILE = "inputs\\wojnicz.txt"
REAL_INPUT_FILE = "inputs\\pan-tadeusz.txt"

In [4]:
# def function: save list of strings to file, separated by newlines
def save_to_file(input: List[str], path: str):
    file = open(path, "w")
    for line in input[:-1]:
        file.write(line + "\n")
    file.write(input[-1])
    file.close()

## Cleaning the text file
Just like in Zipf's law analysis, we have to clean the text file properly.
The difference is, now we have to keep all the punctation marks,
which in Voynich's Manuspcript case means that we have to keep
all the dots (represented as equality signs) separating each
sentence.

Output is saved to <code>cleaned_with_punc.txt</code>

In [5]:
# open input text
# https://www.ic.unicamp.br/~stolfi/voynich/mirror/reeds/docs/FSG.txt
text = open(WOJNICZ_INPUT_FILE, 'r').read().split("\n")

In [6]:
# remove lines with no text
parsed = []

for line in text:
    if line == "":
        continue
    if line == "\x0c":
        continue
    if line.startswith("#"):
        continue
    if not re.search(VALID_CHAR_PUNC_REGEX, line):
        continue

    parsed.append(line)

In [7]:
# remove dashes from end of each line
cleaned = []

for line in parsed:
    # last valid char position
    endpos = 0
    for i, char in enumerate(line):
        if re.match(VALID_CHAR_PUNC_REGEX, char) or char == '=':
            endpos = i

    if line[endpos] == '=':
        cleaned.append(line)
    else:
        cleaned.append(line[:i])

In [8]:
def plot_occ(input):
    # generate occurance count graph

    labels = []
    sizes = []
    for x in input.keys():
        sizes.append(input[x])

    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
    # save plot to file and show
    #if filename == "":
    #    filename = datetime.now().strftime("%Y%m%d-%H%M%S")
    #plt.savefig("outputs\\occurance_" + filename + ".png")
    plt.show()

In [9]:
def count_percentage(input):
    ret_dict = {}
    total_count = 0
    for x in input.keys():
        total_count += input[x]
    for x in input.keys():
        ret_dict[x] = input[x]/total_count

    return ret_dict

In [10]:
# save file
save_to_file(cleaned, "cleaned_with_punc.txt")

## Extracting valid words separated with punctuation
Similarly to Zipf's law analysis, we have to properly
prepare our words, but this time, without removing
the equality signs servis as a proper punctuation
or the manuscrips.

In [11]:
# extract valid words from each line
words_punc = []
for line in cleaned:
    tokens = line.split(",")
    for word in tokens:
        if re.match("^"+VALID_CHAR_PUNC_REGEX+"*$", word) and len(word)>0:
            if (word[-1]) == '=':
                words_punc.append(word[:-1])
                words_punc.append('=')
            else:
                words_punc.append(word)

In [12]:
# save file
save_to_file(words_punc, "words_punc.txt")

In [13]:
# get all neighbors from one side of a word
def get_side(words, index, side):
    ret_list = []

    for i in range(1,3):
        if 0 <= index+side*i < len(words):
            if words[index+side*i] == '=':
                return ret_list
            else:
                ret_list.append(words[index+side*i])

    return ret_list

# get all neighbors of a word
def get_neighbors(words, index):
    ret_list = []
    left = get_side(words, index, -1)
    left.reverse()
    ret_list.extend(left)
    right = get_side(words, index, 1)
    ret_list.extend(right)
    return ret_list

## Grouping the neighbors for each word in text
Next, we iterate through every word in our text to get its neighbors
and add them to the dictionary. In our dictionary, words serves as
keys and key has a list of every neighboring word occurring in text.
We also have to make sure that we don't include any word more than
once.

In [14]:
# create a dictionary of neighbors
neighbors = {}
i = 0
for word in words_punc:
    new_words = get_neighbors(words_punc, i)
    if word in neighbors:
        neighbors[word].extend(new_words)
        neighbors[word] = list(set(neighbors[word]))
    else:
        neighbors[word] = new_words
    i = i+1


In [15]:
## show neighbors of each word
neighbors

{'FGAG2': ['GDAE', 'AR'],
 'GDAE': ['AR',
  'TAE',
  'OHAR',
  '8AET8G',
  '8AM',
  'GDAR',
  'ODAN',
  'FGAG2',
  '4ODG',
  'OEG',
  'GHAM',
  '4ODAE8AR',
  '4ODZG',
  '8G8AN',
  'OEDAM',
  '4OHOEG',
  '8AE',
  'OE',
  'ODAEAE',
  'T8G'],
 'AR': ['2',
  'ARN',
  '8AIR',
  'SAR',
  '4ODC8G',
  'OHAR',
  'HZCG',
  '8AE8G',
  'GHOR',
  'GHCCG',
  'GHO2',
  'G8G',
  'TDZG',
  'THZ',
  'TORAEG',
  'DCCOE',
  'GTCOR',
  'AEDCG',
  'GDAR',
  'OE8AR',
  'GHASG',
  '4OR',
  'DCO8G',
  'TCO',
  'OA2',
  '8AEG',
  'G',
  'DCOE',
  'OROE',
  'GDCCG',
  'OEDCCTG',
  '8AN',
  'ODC2',
  'RAM',
  'TO',
  'OEDC8G',
  'OHTCG',
  'DZCO2',
  'EOEDCOE',
  '8AE',
  '4O8G',
  'ODCAE',
  'R',
  'SCCDG',
  'GDCC8ETC8G',
  'T8G',
  'DAE',
  'TOR',
  'THAM',
  'AR',
  'TAE',
  'OER',
  'GDAE',
  'TG',
  'G8AN',
  'OHCOE',
  'HOR',
  'SOE',
  'TDZ8G',
  'OF',
  'ODAN',
  'ODCCCG',
  'TCAR',
  'OPTC2',
  'AIM',
  'SC8G',
  'ARG',
  '4ODG',
  'OHC8G',
  'OR',
  'T2',
  'GDT8G',
  'AEOK',
  'OHAEAR',
  'TCCG',
  'G

We have to clean our dictionary to remove punctuation marks counted
as words as well as words with empty lists of neighbors.

In [16]:
## clean the dictionary
to_pop = []
for word in neighbors.keys():
    if re.match(PUNCT_REGEX ,word) or neighbors[word] == []:
        to_pop.append(word)

for word in to_pop:
    neighbors.pop(word)

## Counting each word's possible neighbors
When we already extracted a list of different
neighbors for each word, we can easily count
how many unique neighbors each word does have
and sort  them.

In [17]:
## count unique neighbors for each word
neighbor_count = {}
for word in neighbors.keys():
    neighbor_count[word] = len(neighbors[word])

neighbor_count = dict(sorted(neighbor_count.items(), key=lambda item: item[1], reverse=True))

neighbor_count

{'8AM': 794,
 'TOE': 409,
 'TOR': 340,
 '8G': 298,
 '8AR': 297,
 'SOE': 274,
 'TG': 249,
 '8AE': 240,
 'OR': 238,
 'HZG': 225,
 'SO': 225,
 '2': 214,
 'TCG': 202,
 'SOR': 201,
 '8AN': 192,
 'SG': 191,
 'AR': 179,
 'OE': 166,
 'AM': 166,
 'TC8G': 163,
 '8AK': 147,
 '2AM': 147,
 'ODG': 146,
 'ODAM': 144,
 'SCG': 140,
 'T8G': 139,
 '4ODTG': 131,
 '8OE': 130,
 '4OHTG': 130,
 'TDZG': 128,
 'OHG': 126,
 'TO8G': 125,
 '8OR': 122,
 'SC8G': 120,
 'HZOE': 117,
 'HZOR': 112,
 'G': 111,
 '8AIR': 110,
 'ODAE': 110,
 '4ODG': 110,
 'TCOR': 110,
 'TAR': 107,
 '4ODC8G': 105,
 'DZG': 103,
 'TAM': 102,
 'TO': 101,
 'OHAM': 100,
 '4ODAM': 98,
 'R': 98,
 'ODAR': 95,
 'OHTG': 94,
 'TCDG': 90,
 '4ODAR': 90,
 '4OHG': 89,
 'TCOE': 88,
 'O8AM': 86,
 'DTG': 85,
 'OHOE': 82,
 '4ODOE': 82,
 'ODC8G': 82,
 'HZCG': 81,
 'DAM': 81,
 'HTG': 81,
 'K': 81,
 '8': 79,
 'OHAE': 77,
 '2G': 76,
 'TOHG': 75,
 'SO8G': 74,
 '8TG': 74,
 'SCCG': 74,
 'OHTOE': 73,
 'TO8AM': 72,
 'ODOE': 69,
 'TODG': 68,
 '8TOR': 67,
 'ODTG': 66,
 '

## Checking the values for another text written in real-world language

In [18]:
# load text from file
text = open(REAL_INPUT_FILE, 'r', encoding="utf8").read().replace("\n", " ")

VALID_CHAR_PUNC_REGEX = "[a-zA-Z0-9/=ąćęłńóśżźĄĆĘŁŃÓŚŻŹáčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽÄäÖöÜüẞß]"
PUNCT_REGEX = "[.!?()=]"

In [19]:
# convert text to list of words
parsed = ""

for char in text:
    if re.match(PUNCT_REGEX, char):
        parsed+= " = "
    elif char == ' ' or re.match(VALID_CHAR_PUNC_REGEX, char):
        parsed += char

parsed = parsed.split(" ")
words = []


for i, word in enumerate(parsed):
    if re.match("^"+VALID_CHAR_PUNC_REGEX+"+$", word):
            words.append(word.upper())

In [20]:
# save file
save_to_file(words, "words_real_punc.txt")

In [21]:
# create a dictionary of neighbors
neighbors = {}
i = 0
for word in words:
    new_words = get_neighbors(words, i)
    if word in neighbors:
        neighbors[word].extend(new_words)
        neighbors[word] = list(set(neighbors[word]))
    else:
        neighbors[word] = new_words
    i = i+1

In [22]:
## show neighbors of each word
neighbors

{'LITWO': [],
 '=': ['BIALI',
  'WIEKIEM',
  'PROSZĘ',
  'PRAWDAĆ',
  'ŚMIECIE',
  'TRZYMANO',
  'NICZYM',
  'KŁÓTNIA',
  'MU',
  'ROZKOSZNE',
  'CZAMARCE',
  'OBECNI',
  'PIĘŚCI',
  'ZATRWOŻYĆ',
  'ŚPIEWAŁ',
  'MOSKALE',
  'OKO',
  'PODKOWY',
  'DOJŚĆ',
  'PANICZ',
  'KĘDYŚ',
  'WIECZERZĘ',
  'MILCZY',
  'BOŻE',
  'WASALÓW',
  'SŁUCHAJĄC',
  'FAJKĘ',
  'MURAWĄ',
  'UCICHLI',
  'SYPNĘŁA',
  'MYŚLIŁA',
  'TYSIĄC',
  'WYMYŚLIŁEM',
  'A',
  'LECI',
  'GERWAZEGO',
  'ŻARŁA',
  'DOŚCIGNĄĆ',
  'JĘKNĘLI',
  'SADZA',
  'PODARTY',
  'ZNISZCZYŁ',
  'BYĆ',
  'ZAGRAJŻE',
  'BREJGELA',
  'POSĘPNE',
  'ŻONY',
  'WTENCZAS',
  'ANGIELSKU',
  'WRÓCIŁBY',
  'CHOWAŁ',
  'STRZELCÓW',
  'POPRAWI',
  'BŁOGIE',
  'NAD',
  'INSTYNKTOWĄ',
  'CHŁOPIĘ',
  'PANOWIE',
  'MOŻE',
  'PIORUNEM',
  'PAMIĄTEK',
  'BYSTRY',
  'REVERENDISSIME',
  'KRZEWINY',
  'GNIAZDAMI',
  'PRZERWA',
  'DUMANIU',
  'WYPILI',
  'DOTYKALNA',
  'MYŚLIWI',
  'NAJWYRAŹNIEJ',
  'UMIEM',
  'SEJMIKÓW',
  'NASTĘPNIE',
  'BRATY',
  'SZCZĘŚCIE',
 

In [23]:
## clean the dictionary
to_pop = []
for word in neighbors.keys():
    if re.match(PUNCT_REGEX ,word) or neighbors[word] == []:
        to_pop.append(word)

for word in to_pop:
    neighbors.pop(word)

In [24]:
## count unique neighbors for each word
neighbor_count = {}
for word in neighbors.keys():
    neighbor_count[word] = len(neighbors[word])

neighbor_count = dict(sorted(neighbor_count.items(), key=lambda item: item[1], reverse=True))

neighbor_count

{'I': 4570,
 'W': 3396,
 'SIĘ': 3065,
 'Z': 2969,
 'NA': 2818,
 'NIE': 1767,
 'JAK': 1483,
 'DO': 1309,
 'A': 1263,
 'ŻE': 1223,
 'TO': 1023,
 'O': 817,
 'PO': 788,
 'ZA': 788,
 'JUŻ': 746,
 'TAK': 701,
 'OD': 666,
 'CO': 608,
 'GDY': 515,
 'LECZ': 513,
 'BO': 483,
 'JEST': 456,
 'ALE': 447,
 'JAKO': 444,
 'GO': 423,
 'BYŁ': 420,
 'TYLKO': 418,
 'MU': 383,
 'PAN': 378,
 'NIM': 375,
 'JA': 369,
 'POD': 363,
 'MNIE': 356,
 'RZEKŁ': 354,
 'DLA': 344,
 'TAM': 344,
 'TU': 341,
 'U': 335,
 'NAD': 332,
 'TEN': 328,
 'ICH': 327,
 'ON': 323,
 'AŻ': 321,
 'CZY': 304,
 'ZE': 301,
 'PRZY': 297,
 'PRZEZ': 295,
 'PRZED': 284,
 'WIĘC': 281,
 'JESZCZE': 277,
 'BEZ': 271,
 'SĘDZIA': 271,
 'HRABIA': 270,
 'GDZIE': 268,
 'TYM': 259,
 'KU': 259,
 'JEGO': 258,
 'JEJ': 257,
 'SAM': 253,
 'CHOĆ': 253,
 'JĄ': 251,
 'POTEM': 245,
 'MIAŁ': 237,
 'TADEUSZ': 236,
 'DZIŚ': 234,
 'WSZYSCY': 233,
 'KTO': 229,
 'MI': 226,
 'KTÓRY': 221,
 'OCZY': 218,
 'TERAZ': 218,
 'JAKBY': 217,
 'BY': 215,
 'KIEDY': 214,
 'WOJSKI':